In [1]:
!pip install -q git+https://github.com/tensorflow/docs

In [2]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [3]:
!pip install mediapipe opencv-python
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# skeleton styling
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 55.6 MB/s 


DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)

In [4]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 1000

MAX_SEQ_LENGTH = 500
NUM_FEATURES = 2048


In [5]:
dirpath = "/content/drive/MyDrive/yunghab/suwha/"
video_list  = os.listdir("/content/drive/MyDrive/yunghab/suwha")
video_list

['0000_뻐기다.mp4',
 '0001_답.mp4',
 '0002_그냥,그저.mp4',
 '0003_대립.mp4',
 '0004_신선하다,새롭다.mp4',
 '0005_오세아니아.mp4',
 '0006_황홀,휘황하다,휘황찬란하다.mp4',
 '0007_못생기다,못나다,추하다,밉다.mp4',
 '0008_잃어버리다,분실,잃다.mp4',
 '0009_식장.mp4',
 '0010_홍콩.mp4',
 '0011_대답.mp4',
 '0012_신호등.mp4',
 '0013_보물,보석,보화.mp4',
 '0014_전달,전갈,전하다.mp4',
 '0015_터뜨리다,폭발,폭파.mp4',
 '0016_레바논.mp4',
 '0017_화산.mp4',
 '0018_보석상.mp4',
 '0019_겹치다.mp4',
 '0020_장군.mp4',
 '0021_축사.mp4',
 '0022_원래.mp4',
 '0023_식,축하.mp4',
 '0024_(화학)실험.mp4',
 '0025_기금.mp4',
 '0026_유행.mp4',
 '0027_신문사.mp4',
 '0028_신문.mp4',
 '0029_오해.mp4',
 '0030_반짝.mp4',
 '0031_세례.mp4',
 '0032_원금,자금,원가.mp4',
 '0033_전과.mp4',
 '0034_고백,고해.mp4',
 '0035_맥주.mp4',
 '0036_전통.mp4',
 '0037_등대.mp4',
 '0038_빵.mp4',
 '0039_선전.mp4',
 '0040_대화,대담.mp4',
 '0041_별.mp4',
 '0042_불평.mp

In [6]:
video_name = [ list.split("_")[1] for list in video_list]
video_name1 = video_list[:]
video_name2 = video_list[50:]
tag = [ list.split(".")[0] for list in video_name]
tag1 = tag[:]
tag2 = tag[50:]
train_df = pd.DataFrame(data = {
    'video_name':video_name1,
    'tag': tag1
})
test_df = pd.DataFrame(data = {
    'video_name':video_name2,
    'tag': tag2
})

train_df.head(10)
test_df.head(10)


,video_name,tag
0,0050_새어머니.mp4,새어머니
1,0051_새아버지.mp4,새아버지
2,0052_빵집.mp4,빵집
3,"0053_광,광선,라이트,빛,조명,비추다,비치다...","광,광선,라이트,빛,조명,비추다,비치다"
4,0054_신도시.mp4,신도시
5,0055_혁신.mp4,혁신
6,"0056_게우다,구토,토하다.mp4","게우다,구토,토하다"


In [7]:


print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


Total videos for training: 57
Total videos for testing: 7


In [8]:
test_df.shape


(7, 2)

In [9]:

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

'''
# 원본
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    print("load_video path : ", path)
    cap = cv2.VideoCapture(path)
    
    frames = []
    frame_count = 0
    try:
        while True:
            
            ret, frame = cap.read()
            frame_count += 1
            if frame_count < 40 : continue

            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)
'''

# skeleton 추가
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    print("load_video path : ", path)
    cap = cv2.VideoCapture(path)
    
    frames = []
    frame_count = 0
    
    try:

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
          while True:
                
              ret, frame = cap.read()
              frame_count += 1
              if frame_count < 40 : continue

              if not ret:
                  break
              
              # Recolor Feed
              image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              # Make Detections
              results = holistic.process(image)
              # print(results.face_landmarks)
              
              # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
              
              # Recolor image back to BGR for rendering
              image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
              # 2. Right hand
              mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                      mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                      )

              # 3. Left Hand
              mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                      mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                      )

              # 4. Pose Detections
              mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                      )


              frame = crop_center_square(image)
              frame = cv2.resize(frame, resize)
              frame = frame[:, :, [2, 1, 0]]
              frames.append(frame)
              if len(frames) == max_frames:
                  break
    finally:
        cap.release()
    return np.array(frames)

In [10]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87924736/87910968 [==============================] - 1s 0us/step


In [11]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['(화학)실험', '게우다,구토,토하다', '겹치다', '고발,고자질,일러바치다', '고백,고해', '광,광선,라이트,빛,조명,비추다,비치다', '그냥,그저', '근본,기반,기본,기초,바탕,뿌리', '기금', '날리다', '답', '대답', '대립', '대화,대담', '등대', '레바논', '맥주', '못생기다,못나다,추하다,밉다', '반짝', '별', '보물,보석,보화', '보석상', '본질,본성', '불평', '빵', '빵집', '뻐기다', '새아버지', '새어머니', '새해', '선전', '세례', '식,축하', '식장', '신기록', '신도시', '신문', '신문사', '신선하다,새롭다', '신호등', '오세아니아', '오해', '원금,자금,원가', '원래', '유행', '잃어버리다,분실,잃다', '장군', '전과', '전달,전갈,전하다', '전통', '축사', '터뜨리다,폭발,폭파', '행사장', '혁신', '홍콩', '화산', '황홀,휘황하다,휘황찬란하다']


In [12]:
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")

def prepare_all_videos(df):
    import time
    start = time.time()  # 시작 시간 저장
 
 
 
    max_mask = 0
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    video_paths = ["/content/drive/MyDrive/yunghab/suwha/"+path for path in video_paths]
    print(video_paths)
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )
    #frame_copy = np.zeros((252, 224, 224, 3))
    

    # For each video.
    for idx, path in enumerate(video_paths):
        print("진행상황 : " + str(idx + 1) + " / " + str(num_samples))
        
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(path))
        print("frames.shape : ", frames.shape)
        print(type(frames))
        
       
        frames = frames[None, ...]
        
        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            
            video_length = batch.shape[0]
            if max_mask < video_length:
              max_mask = video_length
            video_length
            length = min(MAX_SEQ_LENGTH, video_length)
            print("video_length : ", video_length, " length : ", length)
            for j in range(length):
                
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked
            
            #print('temp_frame_features : ', temp_frame_features.shape)  # (1, 108, 224, 224, 3) -> (1, 20, 2048) 로 만들어 준다.
            #print('temp_frame_mask : ', temp_frame_mask.shape)          # (1, 20)
            

        #print('temp_frame_features2 : ', temp_frame_features.shape)
        #print('temp_frame_mask2 : ', temp_frame_mask.shape)
        #print(temp_frame_mask)
        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()
        print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    print("총 걸린시간 time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    print("max_mask : ", max_mask)
    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df)
test_data, test_labels = prepare_all_videos(test_df)

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")


['/content/drive/MyDrive/yunghab/suwha/0000_뻐기다.mp4', '/content/drive/MyDrive/yunghab/suwha/0001_답.mp4', '/content/drive/MyDrive/yunghab/suwha/0002_그냥,그저.mp4', '/content/drive/MyDrive/yunghab/suwha/0003_대립.mp4', '/content/drive/MyDrive/yunghab/suwha/0004_신선하다,새롭다.mp4', '/content/drive/MyDrive/yunghab/suwha/0005_오세아니아.mp4', '/content/drive/MyDrive/yunghab/suwha/0006_황홀,휘황하다,휘황찬란하다.mp4', '/content/drive/MyDrive/yunghab/suwha/0007_못생기다,못나다,추하다,밉다.mp4', '/content/drive/MyDrive/yunghab/suwha/0008_잃어버리다,분실,잃다.mp4', '/content/drive/MyDrive/yunghab/suwha/0009_식장.mp4', '/content/drive/MyDrive/yunghab/suwha/0010_홍콩.mp4', '/content/drive/MyDrive/yunghab/suwha/0011_대답.mp4', '/content/drive/MyDrive/yunghab/suwha/0012_신호등.mp4', '/content/drive/MyDrive/yunghab/suwha/0013_보물,보석,보화.mp4', '/content/drive/MyDrive/yunghab/suwha/0014_전달,전갈,전하다.mp4', '/content/drive/MyDrive/yunghab/suwha/0015_터뜨ᄅ

In [13]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    seq_model.summary()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 500, 2048)]  0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 gru (GRU)                      (None, 500, 16)      99168       ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
                                                                                                  
 gru_1 (GRU)                    (None, 8)            624         ['gru[0][0]']                

In [14]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(dirpath + path)
    frame_features, frame_mask = prepare_single_video(frames)
    print('frame_features.shape : ', frame_features.shape)
    print('frame_mask.shape : ', frame_mask.shape)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")


test_video = np.random.choice(train_df["video_name"].values.tolist())
print(test_video)
# test_video = [  dirpath + path for path in test_video]

print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

Output hidden; open in https://colab.research.google.com to view.

In [15]:
test_video = np.random.choice(train_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:])

Output hidden; open in https://colab.research.google.com to view.

In [16]:
test_video = np.random.choice(train_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

Output hidden; open in https://colab.research.google.com to view.

In [17]:
test_video = np.random.choice(train_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

Output hidden; open in https://colab.research.google.com to view.

In [18]:
test_video = np.random.choice(train_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

Output hidden; open in https://colab.research.google.com to view.